In [1]:
import sqlite3 as lite # 2.6.0
import sys

# Création de la SGBD et fonctions liées

## Connection

In [2]:
con = lite.connect('bdd.db') # connect

cur = con.cursor() # cursor

tables_names = ['acheteur', 'nature', 'forme_prix', 'procedure', 'transactions', 'titulaire', 
          'lieu', 'type_transaction', 'modification' ]

## Fonction utiles

In [3]:
# doc for the following functions
def tables_available(cur : lite.Cursor) -> list[str]:
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cur.fetchall()
    return tables

def del_tables(cur : lite.Cursor, tables : list[str]) -> None:
    for i in tables:
        cmd = "DROP TABLE IF EXISTS "+i+";"
        cur.execute(cmd)
    print("Tables deleted")
    # test
    t = tables_available(cur)
    if len(t) != 0:
        print("Error : tables not deleted")

def print_tables(cur : lite.Cursor) -> None:
    t = tables_available(cur)
    print("Tables available in the database : (",len(t),"tables )")
    for i in t:
        print(" -",i[0])

In [4]:
def run_sql_script(cur : lite.Cursor, script_path : str) -> None:
    script = open(script_path, 'r').read()
    cur.executescript(script)
    con.commit()

def create_tables(cur : lite.Cursor) -> None:
    run_sql_script(cur, 'create_tables.sql')

def reset_bdd():
    del_tables(cur, tables_names)
    create_tables(cur)

In [5]:
# create_tables(cur)
print_tables(cur)

Tables available in the database : ( 9 tables )
 - acheteur
 - nature
 - forme_Prix
 - procedure
 - transactions
 - titulaire
 - lieu
 - type_transaction
 - modification


# Extracting CSV et processing vers la BDD 
(hors attributs traités par llm pour l'instant)

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv("decp-v3-marches-valides.csv", sep=";", low_memory=False)

In [7]:
df

,titulaire_denominationSociale_1,titulaire_id_1,titulaire_typeIdentifiant_1,titulaire_denominationSociale_2,titulaire_id_2,titulaire_typeIdentifiant_2,titulaire_denominationSociale_3,titulaire_id_3,titulaire_typeIdentifiant_3,procedure,...,origineFrance,updated_at,booleanModification,objetModification,dureeMoisModification,titulairesModification,datePublicationDonneesModification,montantModification,dateNotificationModification,idModification
0,AL RENOV,8.489345e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Procédure adaptée,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CABINET MERLIN,4.286344e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Procédure négociée avec mise en concurrence pr...,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EIFFAGE ENERGIE SYSTEMES CLEVIA EST,3.400232e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Appel d'offres ouvert,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FIELDTURF TARKETT,4.528352e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Procédure adaptée,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EP3,4.801318e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Procédure adaptée,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758116,PHILIPPE HAMELIN,4.991441e+13,SIRET,ARTELIA VILLE ET TRANSPORT,4.445235e+13,SIRET,TRAITCLAIR,4.825308e+13,SIRET,Procédure négociée avec mise en concurrence pr...,...,NaN,NaN,1.0,Modification montants/durée no1 suite à l'aven...,22.0,NaN,NaN,NaN,NaN,NaN
758117,BSO,8.136732e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Appel d'offres ouvert,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758118,VIABILISER.COM,5.032073e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Procédure adaptée,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758119,PERRIER,4.530058e+13,SIRET,NaN,NaN,NaN,NaN,NaN,NaN,Procédure adaptée,...,NaN,NaN,1.0,Travaux supplémentaires,NaN,NaN,2020-09-28,39675.0,2019-11-26,NaN


In [8]:
# get first element of df and print "titulaire_typeIdentifiant_1"
print( df.iloc[0]["titulaire_id_2"] )

# see how much market have a montant of 1
print("Number of market with a montant of 1 :", len(df[df["montant"] == 1]))
print("Percentage of market with a montant of 1 :", len(df[df["montant"] == 1]) / len(df) * 100, "%")

# same with market with a booleanModification set to True
print("Number of market with a booleanModification set to True :", len(df[df["booleanModification"] == True]))
print("Percentage of market with a booleanModification set to True :", len(df[df["booleanModification"] == True]) / len(df) * 100, "%")

# 

nan
Number of market with a montant of 1 : 2877
Percentage of market with a montant of 1 : 0.3794908728290075 %
Number of market with a booleanModification set to True : 95512
Percentage of market with a booleanModification set to True : 12.59851659563579 %


In [9]:
# from those which have a booleanModification set to True, how many have a durationModification of 0
df_modif = df[df["booleanModification"] == True]
# how many is different from NaN
df_modif_duree_notnull = df_modif[df_modif["dureeMoisModification"].notna()]
print("Number of market with a booleanModification set to True :", len(df_modif_duree_notnull))
# mean of dureeMoisModification
print("Mean of dureeMoisModification :", df_modif_duree_notnull["dureeMoisModification"].mean())
# median of dureeMoisModification
print("Median of dureeMoisModification :", df_modif_duree_notnull["dureeMoisModification"].median())

### on the montantModification, how many is different from NaN
df_modif_montant_notnull = df_modif[df_modif["montantModification"].notna()]
print("Number of market with a booleanModification set to True :", len(df_modif_montant_notnull))
# mean of montantModification and median
print("Mean of montantModification :", df_modif_montant_notnull["montantModification"].mean())
print("Median of montantModification :", df_modif_montant_notnull["montantModification"].median())
# create a vector of ratioMontantModification = montantModification / montant
import numpy as np

arr_montant_modif = df_modif_montant_notnull["montantModification"].to_numpy()
arr_montant = df_modif_montant_notnull["montant"].to_numpy()
arr_ratio = arr_montant_modif / arr_montant
# mean of ratioMontantModification
print("Mean of ratioMontantModification :", arr_ratio.mean())
print("Median of ratioMontantModification :", np.median(arr_ratio))



Number of market with a booleanModification set to True : 48076
Mean of dureeMoisModification : 716.9747275147682
Median of dureeMoisModification : 20.0
Number of market with a booleanModification set to True : 65936
Mean of montantModification : 34533897.31467089
Median of montantModification : 160000.0
Mean of ratioMontantModification : 204.10493425222845
Median of ratioMontantModification : 1.0247109100050276


In [10]:
# list all the columns of the dataframe
print("Columns of the dataframe :")
# create a file where to write the columns
#f = open("columns.txt", "w")
for i in df.columns:
    print(" -", i)
    #f.write(i+"\n")
#f.close()

Columns of the dataframe :
 - titulaire_denominationSociale_1
 - titulaire_id_1
 - titulaire_typeIdentifiant_1
 - titulaire_denominationSociale_2
 - titulaire_id_2
 - titulaire_typeIdentifiant_2
 - titulaire_denominationSociale_3
 - titulaire_id_3
 - titulaire_typeIdentifiant_3
 - procedure
 - nature
 - codeCPV
 - dureeMois
 - id
 - formePrix
 - dateNotification
 - objet
 - montant
 - acheteur.id
 - source
 - acheteur.nom
 - lieuExecution.code
 - lieuExecution.typeCode
 - lieuExecution.nom
 - datePublicationDonnees
 - created_at
 - modaliteExecution
 - marcheInnovant
 - ccag
 - offresRecues
 - attributionAvance
 - typeGroupementOperateurs
 - TypePrix
 - considerationsSociales
 - considerationsEnvironnementales
 - sousTraitanceDeclaree
 - actesSousTraitance
 - modificationsActesSousTraitance
 - technique
 - idAccordCadre
 - tauxAvance
 - origineUE
 - origineFrance
 - updated_at
 - booleanModification
 - objetModification
 - dureeMoisModification
 - titulairesModification
 - datePublicat

In [11]:
# see if there are element with titulaire_id_1 with NaN
df_titulaire_id_1_notnull = df[df["titulaire_id_1"].notna()]
print("Number of market with a titulaire_id_1 not null :", len(df_titulaire_id_1_notnull))
print("Percentage of market with a titulaire_id_1 not null :", len(df_titulaire_id_1_notnull) / len(df) * 100, "%")

Number of market with a titulaire_id_1 not null : 755952
Percentage of market with a titulaire_id_1 not null : 99.71389791339377 %


In [12]:
# print df procedure
print("Unique values of the procedure column :")
df_procedure_unique = df["procedure"].unique()
# show the unique values
for i in df_procedure_unique:
    print(" -", i)

# querry all the rows which have a nan value in the procedure column (with all the columns)
df_procedure_nan = df[df["procedure"].isna()]

print("\nNumber of nan in the procedure column :", len(df_procedure_nan))
print("Percentage of nan in the procedure column :", len(df_procedure_nan) / len(df) * 100, "%")

""" print the "objet" attribute of the first 10 which have a nan value
print("Objet of the first 10 which have a nan value in the procedure column :")
df_procedure_nan_objet = df_procedure_nan["objet"]
for i in range(50,60):
    print(" -", df_procedure_nan_objet.iloc[i])
     """;

Unique values of the procedure column :
 - Procédure adaptée
 - Procédure négociée avec mise en concurrence préalable
 - Appel d'offres ouvert
 - nan
 - Procédure avec négociation
 - Procédure concurrentielle avec négociation
 - Marché négocié sans publicité ni mise en concurrence préalable
 - Marché public négocié sans publicité ni mise en concurrence préalable
 - Marché passé sans publicité ni mise en concurrence préalable
 - Appel d'offres restreint
 - Dialogue compétitif
 - NC
 - ProcÃ©dure adaptÃ©e

Number of nan in the procedure column : 25845
Percentage of nan in the procedure column : 3.4090864123273197 %


In [39]:
# print the first 10 value in the "codeCPV" column

# df_codeCPV = df["lieuExecution.code"]
# print("len lieu Execution.code:",len(df_codeCPV.unique()) )

""" df_codeCPV = df[["lieuExecution.nom", "lieuExecution.code"]]
# shape of the dataframe
print("Shape of the dataframe :")
print(df_codeCPV.shape)
print("shape unique entries:", df_codeCPV.drop_duplicates().shape)

df_codeCPV = df[["lieuExecution.nom", "lieuExecution.code", "lieuExecution.typeCode"]]
# shape of the dataframe
print("shape unique entries:", df_codeCPV.drop_duplicates().shape) """

#print(len(df_codeCPV.unique()))
""" import unidecode
#remove nan
df_codeCPV = df["lieuExecution.typeCode"].dropna()

arr = df_codeCPV.unique().astype(str)

arr = np.char.lower(arr)
arr = np.char.replace(arr, '!', '')
cleaned_arr = np.array([unidecode.unidecode(word) for word in arr])
np.unique(cleaned_arr)
 """
#print("First 10 values of the codeCPV column :")
#for i in range(10):
#    print(" -", df_codeCPV.iloc[i])




# print the one with a len of 255
# print("CodeCPV with a length of 255 :")
# df_codeCPV_255 = df_codeCPV[df_codeCPV.str.len() == 54]
# print(df_codeCPV_255.iloc[0])
# print first 1
# print(" -", df_codeCPV_255.iloc[0])

df_codeCPV = df["modificationsActesSousTraitance"]
# get the one with the max length and print its index
#print("max length of titulairesModification : ", df_codeCPV.str.len().argmax())

df_codeCPV.dropna().unique()


#df_codeCPV.dropna()
#print("max length of codeCPV : ", df_codeCPV.str.len().max())
#print("min length of codeCPV : ", df_codeCPV.str.len().min())
#print("mean length of codeCPV : ", df_codeCPV.str.len().mean())

# res = res[res == True]
# # dropna and all true
# len(res)

#df_codeCPV.dropna().shape
#df_codeCPV.dropna().unique().shape



array(['[]',
       "[{'dureeMois': 1, 'montant': 0, 'id': 2, 'dateNotificationModificationActeSousTraitance': '2024-01-09', 'datePublicationDonneesModificationActeSousTraitance': '2024-01-16'}]"],
      dtype=object)